In [1]:
import pandas as pd

TrainDF = pd.read_csv('train.csv')
TestDF = pd.read_csv('test.csv')

X = TrainDF.iloc[:,:-1]
X.head()
y = TrainDF.iloc[:,-1]
X.shape

(15120, 55)

In [2]:
TrainDF['soilMax'] =  TrainDF.iloc[:,15:55].idxmax(axis=1)
TrainDF['soil_type'] = TrainDF['soilMax'].map(lambda x: x.lstrip('Soil_Type'))
TrainDF['soil_type'] 

TestDF['soilMax'] =  TestDF.iloc[:,15:55].idxmax(axis=1)
TestDF['soil_type'] = TestDF['soilMax'].map(lambda x: x.lstrip('Soil_Type'))
# TestDF['soil_type'] 

TrainDF['wilderMax'] = (TrainDF.iloc[:,11:15].idxmax(axis=1))
TrainDF['wilderness'] = TrainDF['wilderMax'].map({ 'Wilderness_Area1':0,'Wilderness_Area2':1,'Wilderness_Area3':2,'Wilderness_Area4':3})
# TrainDF['wilderness']

TestDF['wilderMax'] = (TestDF.iloc[:,11:15].idxmax(axis=1))
TestDF['wilderness'] = TestDF['wilderMax'].map({ 'Wilderness_Area1':0,'Wilderness_Area2':1,'Wilderness_Area3':2,'Wilderness_Area4':3})
# TestDF['wilderness']

# Remove id and take only relevant fields

X = TrainDF.loc[:,[ 'Elevation', 'Aspect', 'Slope',
       'Horizontal_Distance_To_Hydrology', 'Vertical_Distance_To_Hydrology',
       'Horizontal_Distance_To_Roadways', 'Hillshade_9am', 'Hillshade_Noon',
       'Hillshade_3pm', 'Horizontal_Distance_To_Fire_Points','wilderness','soil_type']]
X.head()

Xtest = TestDF.loc[:,[ 'Elevation', 'Aspect', 'Slope',
       'Horizontal_Distance_To_Hydrology', 'Vertical_Distance_To_Hydrology',
       'Horizontal_Distance_To_Roadways', 'Hillshade_9am', 'Hillshade_Noon',
       'Hillshade_3pm', 'Horizontal_Distance_To_Fire_Points','wilderness','soil_type']]

y = TrainDF.loc[:,['Cover_Type']]

In [3]:
from keras.utils import np_utils
import numpy
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline


# baseline model
def create_baseline1():
    # create model
    model = Sequential()
    model.add(Dense(12, input_dim=12, kernel_initializer='normal', activation='relu'))
    # With one layer in between, learning not good
    model.add(Dense(7, kernel_initializer='normal', activation='softmax'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# encode class values as integers
encoder = LabelEncoder()
encoder.fit(y)
encoded_Y = encoder.transform(y)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(y)
encoded_Y = encoder.transform(y)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)


from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, dummy_y, 
                                                    train_size=0.9)

Using TensorFlow backend.
/home/a95/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:95: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/a95/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:128: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/a95/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [4]:
model = create_baseline1()
model.fit(x = X_train ,y = y_train, epochs = 500 , batch_size = 1024)

Epoch 1/500
13608/13608 [==============================] - 1s 47us/step - loss: 2.7373 - acc: 0.7605
Epoch 2/500
13608/13608 [==============================] - 0s 3us/step - loss: 1.2411 - acc: 0.7714
Epoch 3/500
13608/13608 [==============================] - 0s 3us/step - loss: 0.8247 - acc: 0.8029
Epoch 4/500
13608/13608 [==============================] - 0s 4us/step - loss: 0.5792 - acc: 0.8289
Epoch 5/500
13608/13608 [==============================] - 0s 3us/step - loss: 0.4717 - acc: 0.8397
Epoch 6/500
13608/13608 [==============================] - 0s 3us/step - loss: 0.4293 - acc: 0.8490
Epoch 7/500
13608/13608 [==============================] - 0s 3us/step - loss: 0.4068 - acc: 0.8546
Epoch 8/500
13608/13608 [==============================] - 0s 4us/step - loss: 0.3967 - acc: 0.8564
Epoch 9/500
13608/13608 [==============================] - 0s 4us/step - loss: 0.3901 - acc: 0.8578
Epoch 10/500
13608/13608 [==============================] - 0s 3us/step - loss: 0.3864 - acc: 0.858

13608/13608 [==============================] - 0s 3us/step - loss: 0.2364 - acc: 0.8903
Epoch 83/500
13608/13608 [==============================] - 0s 3us/step - loss: 0.2368 - acc: 0.8903
Epoch 84/500
13608/13608 [==============================] - 0s 3us/step - loss: 0.2363 - acc: 0.8911
Epoch 85/500
13608/13608 [==============================] - 0s 3us/step - loss: 0.2367 - acc: 0.8900
Epoch 86/500
13608/13608 [==============================] - 0s 3us/step - loss: 0.2357 - acc: 0.8907
Epoch 87/500
13608/13608 [==============================] - 0s 3us/step - loss: 0.2352 - acc: 0.8911
Epoch 88/500
13608/13608 [==============================] - 0s 3us/step - loss: 0.2355 - acc: 0.8906
Epoch 89/500
13608/13608 [==============================] - 0s 3us/step - loss: 0.2351 - acc: 0.8913
Epoch 90/500
13608/13608 [==============================] - 0s 3us/step - loss: 0.2364 - acc: 0.8897
Epoch 91/500
13608/13608 [==============================] - 0s 3us/step - loss: 0.2356 - acc: 0.8910
Epo

13608/13608 [==============================] - 0s 3us/step - loss: 0.2219 - acc: 0.8981
Epoch 163/500
13608/13608 [==============================] - 0s 3us/step - loss: 0.2232 - acc: 0.8971
Epoch 164/500
13608/13608 [==============================] - 0s 3us/step - loss: 0.2237 - acc: 0.8971
Epoch 165/500
13608/13608 [==============================] - 0s 3us/step - loss: 0.2240 - acc: 0.8965
Epoch 166/500
13608/13608 [==============================] - 0s 3us/step - loss: 0.2218 - acc: 0.8984
Epoch 167/500
13608/13608 [==============================] - 0s 4us/step - loss: 0.2207 - acc: 0.8989
Epoch 168/500
13608/13608 [==============================] - 0s 3us/step - loss: 0.2216 - acc: 0.8988
Epoch 169/500
13608/13608 [==============================] - 0s 4us/step - loss: 0.2219 - acc: 0.8974
Epoch 170/500
13608/13608 [==============================] - 0s 4us/step - loss: 0.2230 - acc: 0.8972
Epoch 171/500
13608/13608 [==============================] - 0s 4us/step - loss: 0.2218 - acc: 0

13608/13608 [==============================] - 0s 4us/step - loss: 0.2184 - acc: 0.8985
Epoch 243/500
13608/13608 [==============================] - 0s 3us/step - loss: 0.2183 - acc: 0.8986
Epoch 244/500
13608/13608 [==============================] - 0s 3us/step - loss: 0.2150 - acc: 0.9001
Epoch 245/500
13608/13608 [==============================] - 0s 3us/step - loss: 0.2151 - acc: 0.9007
Epoch 246/500
13608/13608 [==============================] - 0s 3us/step - loss: 0.2144 - acc: 0.9010
Epoch 247/500
13608/13608 [==============================] - 0s 3us/step - loss: 0.2130 - acc: 0.9014
Epoch 248/500
13608/13608 [==============================] - 0s 3us/step - loss: 0.2132 - acc: 0.9017
Epoch 249/500
13608/13608 [==============================] - 0s 3us/step - loss: 0.2129 - acc: 0.9019
Epoch 250/500
13608/13608 [==============================] - 0s 3us/step - loss: 0.2131 - acc: 0.9018
Epoch 251/500
13608/13608 [==============================] - 0s 4us/step - loss: 0.2131 - acc: 0

13608/13608 [==============================] - 0s 3us/step - loss: 0.2090 - acc: 0.9041
Epoch 323/500
13608/13608 [==============================] - 0s 4us/step - loss: 0.2071 - acc: 0.9053
Epoch 324/500
13608/13608 [==============================] - 0s 3us/step - loss: 0.2091 - acc: 0.9038
Epoch 325/500
13608/13608 [==============================] - 0s 3us/step - loss: 0.2090 - acc: 0.9041
Epoch 326/500
13608/13608 [==============================] - 0s 3us/step - loss: 0.2083 - acc: 0.9048
Epoch 327/500
13608/13608 [==============================] - 0s 3us/step - loss: 0.2078 - acc: 0.9038
Epoch 328/500
13608/13608 [==============================] - 0s 3us/step - loss: 0.2070 - acc: 0.9057
Epoch 329/500
13608/13608 [==============================] - 0s 3us/step - loss: 0.2065 - acc: 0.9061
Epoch 330/500
13608/13608 [==============================] - 0s 3us/step - loss: 0.2069 - acc: 0.9058
Epoch 331/500
13608/13608 [==============================] - 0s 3us/step - loss: 0.2095 - acc: 0

13608/13608 [==============================] - 0s 3us/step - loss: 0.2029 - acc: 0.9070
Epoch 403/500
13608/13608 [==============================] - 0s 4us/step - loss: 0.2024 - acc: 0.9076
Epoch 404/500
13608/13608 [==============================] - 0s 3us/step - loss: 0.2029 - acc: 0.9075
Epoch 405/500
13608/13608 [==============================] - 0s 3us/step - loss: 0.2026 - acc: 0.9070
Epoch 406/500
13608/13608 [==============================] - 0s 3us/step - loss: 0.2019 - acc: 0.9088
Epoch 407/500
13608/13608 [==============================] - 0s 3us/step - loss: 0.2019 - acc: 0.9077
Epoch 408/500
13608/13608 [==============================] - 0s 4us/step - loss: 0.2016 - acc: 0.9085
Epoch 409/500
13608/13608 [==============================] - 0s 3us/step - loss: 0.2033 - acc: 0.9067
Epoch 410/500
13608/13608 [==============================] - 0s 4us/step - loss: 0.2024 - acc: 0.9081
Epoch 411/500
13608/13608 [==============================] - 0s 4us/step - loss: 0.2055 - acc: 0

13608/13608 [==============================] - 0s 3us/step - loss: 0.1988 - acc: 0.9097
Epoch 483/500
13608/13608 [==============================] - 0s 3us/step - loss: 0.1982 - acc: 0.9095
Epoch 484/500
13608/13608 [==============================] - 0s 4us/step - loss: 0.1988 - acc: 0.9099
Epoch 485/500
13608/13608 [==============================] - 0s 3us/step - loss: 0.1988 - acc: 0.9095
Epoch 486/500
13608/13608 [==============================] - 0s 3us/step - loss: 0.1983 - acc: 0.9106
Epoch 487/500
13608/13608 [==============================] - 0s 3us/step - loss: 0.2026 - acc: 0.9075
Epoch 488/500
13608/13608 [==============================] - 0s 3us/step - loss: 0.2015 - acc: 0.9062
Epoch 489/500
13608/13608 [==============================] - 0s 4us/step - loss: 0.1988 - acc: 0.9088
Epoch 490/500
13608/13608 [==============================] - 0s 3us/step - loss: 0.1990 - acc: 0.9095
Epoch 491/500
13608/13608 [==============================] - 0s 4us/step - loss: 0.1990 - acc: 0

In [5]:
preds = model.evaluate(x = X_test , y = y_test)

print()
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

1512/1512 [==============================] - 0s 39us/step

Loss = 0.212298198904
Test Accuracy = 0.901738482178


In [6]:
prediction = model.predict(Xtest)

import numpy as np
Test_label_pred = np.argmax(prediction,axis=-1) + 1

Test_label_pred

newDataframe = pd.DataFrame({'Id':TestDF.Id , 'Cover_Type' : Test_label_pred}).set_index('Id').to_csv('ForestRemoveID12ColKeras')

"Accuracy 48.14% Kaggle "

'Accuracy 48.14% Kaggle '